In [110]:
import pandas as pd
import csv
import requests
from bs4 import BeautifulSoup
import requests
import pymongo
import datetime
import json
import requests
import time
from bson import ObjectId

In [116]:
url = 'https://www.worldometers.info/coronavirus/'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)
soup = BeautifulSoup(r.text)

In [117]:
trs = soup.find_all('tr')

In [118]:
# The default port used by MongoDB is 27017
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.corona_db
collection = db.country_coords
coordsList = collection.find_one()['countries']

In [119]:
# Loop through countries
countries = []
for i in range(len(trs)):
    # If country row, pull data
    if 'href="country' in str(trs[i]):
        country = str(trs[i]).split('\n')
        name = country[2].split('</a>')[0].split("/\">")[1]
        totalCases = country[3].split('>')[1].split('<')[0]
        deaths = country[5].split("\">")[1].split('<')[0]
        recovered = country[8].split("\">")[1].split('<')[0]
        activeCases = country[10].split("\">")[1].split('<')[0]
        tests = country[14].split("\">")[1].split('<')[0]
        population = country[16].split('>')[2].split('<')[0]
        
        for x in range(len(coordsList)):
            if coordsList[x]['name'] == name:
#                 print(coordsList[x])
                lat = coordsList[x]['lat']
                lng = coordsList[x]['lng']
                break
        
        
        try:
            newCases = country[4].split('>+')[1].split('</')[0]
        except:
            newCases = 'n/a'
#         print(name,totalCases,deaths,recovered,activeCases,tests,population,newCases)
        
        countryObj = {
            'name': name,
            'totalCases' : totalCases,
            'deaths' : deaths,
            'recovered' : recovered,
            'activeCases' : activeCases,
            'tests' : tests,
            'population' : population,
            'coords' : [lng, lat]
        }
        print(countryObj)
        countries.append(countryObj)
        print('\n')
        i+=1

{'name': 'USA', 'totalCases': '3,946,671', 'deaths': '143,621 ', 'recovered': '1,831,285', 'activeCases': '1,971,765', 'tests': '48,918,363', 'population': '331,102,850', 'coords': [-95.712891, 37.09024]}


{'name': 'Brazil', 'totalCases': '2,102,559', 'deaths': '79,590 ', 'recovered': '1,371,229', 'activeCases': '651,740', 'tests': '4,911,063', 'population': '212,636,496', 'coords': [-51.92528, -14.235004]}


{'name': 'India', 'totalCases': '1,154,917', 'deaths': '28,099 ', 'recovered': '724,702', 'activeCases': '402,116', 'tests': '14,047,908', 'population': '1,380,678,271', 'coords': [78.96288, 20.593684]}


{'name': 'Russia', 'totalCases': '777,486', 'deaths': '12,427 ', 'recovered': '553,602', 'activeCases': '211,457', 'tests': '25,251,614', 'population': '145,937,857', 'coords': [105.318756, 61.52401]}


{'name': 'South Africa', 'totalCases': '364,328', 'deaths': '5,033 ', 'recovered': '191,059', 'activeCases': '168,236', 'tests': '2,471,747', 'population': '59,344,794', 'coords'

{'name': 'Thailand', 'totalCases': '3,249', 'deaths': '58 ', 'recovered': '3,096', 'activeCases': '95', 'tests': '603,657', 'population': '69,809,305', 'coords': [100.992541, 15.870032]}


{'name': 'Zambia', 'totalCases': '3,148', 'deaths': '120 ', 'recovered': '1,497', 'activeCases': '1,531', 'tests': '69,296', 'population': '18,404,710', 'coords': [27.849332, -13.133897]}


{'name': 'Nicaragua', 'totalCases': '3,147', 'deaths': '99 ', 'recovered': '2,282', 'activeCases': '766', 'tests': '', 'population': '6,628,387', 'coords': [-85.207229, 12.865416]}


{'name': 'Somalia', 'totalCases': '3,119', 'deaths': '93 ', 'recovered': '1,457', 'activeCases': '1,569', 'tests': '', 'population': '15,911,122', 'coords': [46.199616, 5.152149]}


{'name': 'Equatorial Guinea', 'totalCases': '3,071', 'deaths': '51 ', 'recovered': '842', 'activeCases': '2,178', 'tests': '16,000', 'population': '1,404,723', 'coords': [10.267895, 1.650801]}


{'name': 'Maldives', 'totalCases': '2,966', 'deaths': '15 ', 

{'name': 'Eswatini', 'totalCases': '1,729', 'deaths': '21 ', 'recovered': '788', 'activeCases': '920', 'tests': '17,559', 'population': '1,160,758', 'coords': [31.465866, -26.522503]}


{'name': 'Sierra Leone', 'totalCases': '1,701', 'deaths': '65 ', 'recovered': '1,237', 'activeCases': '399', 'tests': '', 'population': '7,984,176', 'coords': [-11.779889, 8.460555]}


{'name': 'Benin', 'totalCases': '1,602', 'deaths': '31 ', 'recovered': '782', 'activeCases': '789', 'tests': '72,832', 'population': '12,136,306', 'coords': [2.315834, 9.30769]}


{'name': 'Yemen', 'totalCases': '1,581', 'deaths': '443 ', 'recovered': '701', 'activeCases': '437', 'tests': '120', 'population': '29,854,516', 'coords': [48.516388, 15.552727]}


{'name': 'New Zealand', 'totalCases': '1,550', 'deaths': '22 ', 'recovered': '1,506', 'activeCases': '22', 'tests': '441,123', 'population': '', 'coords': [174.885971, -40.900557]}


{'name': 'Rwanda', 'totalCases': '1,539', 'deaths': '5 ', 'recovered': '819', 'active

In [120]:
countryDict = {
    'countries': countries
}
countryDict

{'countries': [{'name': 'USA',
   'totalCases': '3,946,671',
   'deaths': '143,621 ',
   'recovered': '1,831,285',
   'activeCases': '1,971,765',
   'tests': '48,918,363',
   'population': '331,102,850',
   'coords': [-95.712891, 37.09024]},
  {'name': 'Brazil',
   'totalCases': '2,102,559',
   'deaths': '79,590 ',
   'recovered': '1,371,229',
   'activeCases': '651,740',
   'tests': '4,911,063',
   'population': '212,636,496',
   'coords': [-51.92528, -14.235004]},
  {'name': 'India',
   'totalCases': '1,154,917',
   'deaths': '28,099 ',
   'recovered': '724,702',
   'activeCases': '402,116',
   'tests': '14,047,908',
   'population': '1,380,678,271',
   'coords': [78.96288, 20.593684]},
  {'name': 'Russia',
   'totalCases': '777,486',
   'deaths': '12,427 ',
   'recovered': '553,602',
   'activeCases': '211,457',
   'tests': '25,251,614',
   'population': '145,937,857',
   'coords': [105.318756, 61.52401]},
  {'name': 'South Africa',
   'totalCases': '364,328',
   'deaths': '5,033 ',

In [121]:
# Update data in db
collection = db.corona_data

In [122]:
res = collection.find({"_id" : ObjectId("5f1201553772928119aabb9f")})
for r in res:
    print(r)

{'_id': ObjectId('5f1201553772928119aabb9f'), 'countries': [{'name': 'USA', 'totalCases': '3,737,841', 'deaths': '141,617 ', 'recovered': '1,696,628', 'activeCases': '1,899,596', 'tests': '46,121,093', 'population': '331,086,970', 'coords': [-95.712891, 37.09024]}, {'name': 'Brazil', 'totalCases': '2,021,834', 'deaths': '76,997 ', 'recovered': '1,366,775', 'activeCases': '578,062', 'tests': '4,911,063', 'population': '212,624,130', 'coords': [-51.92528, -14.235004]}, {'name': 'India', 'totalCases': '1,039,045', 'deaths': '26,285 ', 'recovered': '654,056', 'activeCases': '358,704', 'tests': '13,072,718', 'population': '1,380,567,150', 'coords': [78.96288, 20.593684]}, {'name': 'Russia', 'totalCases': '759,203', 'deaths': '12,123 ', 'recovered': '539,373', 'activeCases': '207,707', 'tests': '24,364,568', 'population': '145,937,345', 'coords': [105.318756, 61.52401]}, {'name': 'Peru', 'totalCases': '341,586', 'deaths': '12,615 ', 'recovered': '230,994', 'activeCases': '97,977', 'tests': '

In [123]:
results = collection.find_one()
results

{'_id': ObjectId('5f1201553772928119aabb9f'),
 'countries': [{'name': 'USA',
   'totalCases': '3,737,841',
   'deaths': '141,617 ',
   'recovered': '1,696,628',
   'activeCases': '1,899,596',
   'tests': '46,121,093',
   'population': '331,086,970',
   'coords': [-95.712891, 37.09024]},
  {'name': 'Brazil',
   'totalCases': '2,021,834',
   'deaths': '76,997 ',
   'recovered': '1,366,775',
   'activeCases': '578,062',
   'tests': '4,911,063',
   'population': '212,624,130',
   'coords': [-51.92528, -14.235004]},
  {'name': 'India',
   'totalCases': '1,039,045',
   'deaths': '26,285 ',
   'recovered': '654,056',
   'activeCases': '358,704',
   'tests': '13,072,718',
   'population': '1,380,567,150',
   'coords': [78.96288, 20.593684]},
  {'name': 'Russia',
   'totalCases': '759,203',
   'deaths': '12,123 ',
   'recovered': '539,373',
   'activeCases': '207,707',
   'tests': '24,364,568',
   'population': '145,937,345',
   'coords': [105.318756, 61.52401]},
  {'name': 'Peru',
   'totalCas

In [124]:
results.update({"$set":{"countries":countryDict['countries']}})

In [126]:
collection.update_one({},{"$set":{"countries":countryDict['countries']}})